In [ ]:
#| default_exp live

## Live Transcription

This module implements a real-time speech-to-text system that: <br>
- Listens to microphone input continuously <br>
- Uses **Silero VAD** (Voice Activity Detection) to detect when you are speaking <br>
- Automatically segments audio into utterances based on natural pauses <br>
- Transcribes each utterance using **Faster-Whisper** (optimized Whisper implementation) <br>
- Streams transcribed text chunks as they become available <br>

The system is optimized for interactive applications that need responsive, chunked speech transcription with minimal latency.

### Architecture Overview

The implementation uses a multi-threaded, asynchronous architecture:

1. **PyAudio Thread**: Captures raw audio from the microphone <br>
2. **AsyncIO Queue**: Thread-safe bridge between audio callback and async processing <br>
3. **Processing Loop**: Analyzes audio with VAD and triggers transcription <br>
4. **Worker Threads**: Runs Whisper transcription without blocking the main loop <br>

This design ensures audio capture never blocks or drops frames while maintaining responsive transcription.

## Imports and Device Selection

The module imports only essential dependencies: <br>
- **numpy**: Audio data manipulation and buffer management <br>
- **torch**: Required for Silero VAD model <br>
- **pyaudio**: Cross-platform audio I/O <br>
- **faster_whisper**: Optimized Whisper implementation with CTranslate2 backend <br>
- **asyncio**: Non-blocking audio processing <br>
- **logging**: Debug and status output <br>

### Device Selection Strategy

The `get_device()` helper automatically selects the best available compute device:

1. **CPU (forced)**: If `force_cpu=True` <br>
2. **CUDA**: NVIDIA GPU acceleration (if available) <br>
3. **MPS**: Apple Silicon Metal Performance Shaders (if available) <br>
4. **CPU (fallback)**: Default fallback <br>

For Whisper models, device selection impacts speed significantly: <br>
- CPU: Slower but works everywhere <br>
- CUDA: 5-10x faster on compatible NVIDIA GPUs <br>
- MPS: 3-5x faster on M1/M2/M3 Macs <br>

In [ ]:
#| export
#| include: false
import logging
import asyncio
from typing import Optional, Callable
from queue import Queue

import numpy as np
import pyaudio
import torch
from faster_whisper import WhisperModel

def get_device(force_cpu: bool = False) -> str:
    """Pick best available device."""
    if force_cpu:
        return "cpu"
    if torch.cuda.is_available():
        return "cuda"
    if torch.backends.mps.is_available():
        try:
            torch.mps.empty_cache()
        except Exception:
            pass
        return "mps"
    return "cpu"

def load_silero_vad():
    """Load Silero VAD model from torch hub."""
    try:
        model, _utils = torch.hub.load(
            repo_or_dir='snakers4/silero-vad',
            model='silero_vad',
            force_reload=False,
            onnx=False
        )
        return model
    except Exception as e:
        logging.warning(f"Failed to load Silero VAD: {e}")
        return None

/home/jens/miniconda3/envs/tui_writer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🗣️ Silero VAD (Voice Activity Detection)

This small helper loads the **Silero VAD** model from `torch.hub`.

Silero VAD is a lightweight neural model that outputs the **probability of speech** (0–1).  
We’ll use it to decide when the user is talking or has paused — so we can send only meaningful speech chunks to Whisper.

If Silero fails to load (e.g., offline), we just return `None` and handle it later.


# 🎙️ LiveTranscriber — Real-time Speech-to-Text

The `LiveTranscriber` class captures live audio from your microphone, detects when you’re speaking using **Silero VAD**, and transcribes each spoken sentence using **Faster-Whisper** once you pause.

It runs asynchronously, making it ideal for real-time interfaces like TUIs or assistants.

---

### ⚙️ Parameters
- **`model_id`** – Whisper model to use (e.g. `"tiny"`, `"base"`, `"small"`).  
- **`language`** – Language code for transcription (default `"en"`).  
- **`force_cpu`** – Force CPU usage even if GPU is available.  
- **`on_transcript`** – Callback called with each transcribed text chunk.  
- **`vad_threshold`** – Silero confidence threshold (0.0–1.0, higher = stricter).  
- **`min_speech_duration_ms`** – Minimum length of speech to count as valid.  
- **`min_silence_duration_ms`** – How long silence must last before starting transcription.

---

### 🧠 Main Methods
- **`start()`** — Begins microphone capture and transcription loop (async).  
- **`stop()`** — Gracefully stops audio processing.  
- **`process_audio()`** — Runs continuously, detecting speech/silence and triggering transcription.  
- **`_transcribe_chunk()`** — Uses Whisper to transcribe one full utterance.  
- **`_detect_speech_silero()`** — Returns `True` if Silero VAD detects speech in the current chunk.  

---

### 🔄 Summary
- Audio is streamed in 32 ms chunks (512 samples at 16 kHz).  
- Each chunk is passed to Silero VAD → speech or silence.  
- When silence lasts long enough, the buffered audio is sent to Whisper.  
- The result is sent to your `on_transcript` callback.


In [ ]:
#| export

class LiveTranscriber:
    """
    Improved LiveTranscriber:
      - uses asyncio.Queue for clean async consumption
      - pushes audio from audio thread with loop.call_soon_threadsafe
      - collects chunks in a list (no repeated np.append)
      - does NOT append silence chunks into the sent buffer
      - dispatches transcription to a thread and safely calls callbacks
      - provides stop() for clean shutdown
    """

    def __init__(
        self,
        model_id: str = "openai/whisper-base",
        language: str = "en",
        force_cpu: bool = False,
        on_transcript: Optional[Callable[[str], None]] = None,
        vad_threshold: float = 0.5,
        min_speech_duration_ms: int = 250,
        min_silence_duration_ms: int = 500,
    ):
        self.logger = logging.getLogger(__name__)
        self.on_transcript = on_transcript

        self.model_id = model_id
        self.language = language

        self.sample_rate = 16000

        # ASR model
        self.device = get_device(force_cpu=force_cpu)
        self.transcribe_model = WhisperModel(
            self.model_id,
            device=self.device,
            compute_type="int8" if self.device == "cpu" else "float16",
        )

        # VAD
        self.vad_threshold = vad_threshold
        self.silero_model = load_silero_vad()
        if self.silero_model is None:
            raise RuntimeError("Silero VAD failed to load. Cannot continue.")

        # thresholds in samples
        self.min_speech_samples = int(self.sample_rate * min_speech_duration_ms / 1000)
        self.min_silence_samples = int(self.sample_rate * min_silence_duration_ms / 1000)

        # async queue and runtime vars (set in start())
        self.async_queue: Optional[asyncio.Queue] = None
        self.loop: Optional[asyncio.AbstractEventLoop] = None

        # state used by process_audio
        self.is_running = False
        self.is_speech_active = False
        self.speech_chunks: list[np.ndarray] = []  # collect speech chunks here (no np.append)
        self.speech_samples = 0  # total samples currently in speech_chunks
        self.silence_counter = 0  # in samples

        # Pre-buffer to capture start of speech (keeps last 3 chunks before speech detected)
        self.pre_buffer: list[np.ndarray] = []
        self.pre_buffer_chunks = 3

        self._pyaudio = None
        self._stream = None

        self.logger.info(
            f"LiveTranscriber init (model={model_id}, device={self.device}, rate={self.sample_rate})"
        )

    # -------------------------
    # VAD & transcription helpers
    # -------------------------
    def _detect_speech_silero(self, audio_chunk: np.ndarray) -> bool:
        """Return True if Silero considers this chunk speech."""
        try:
            audio_tensor = torch.from_numpy(audio_chunk).float()
            prob = self.silero_model(audio_tensor, self.sample_rate).item()
            return prob > self.vad_threshold
        except Exception as e:
            self.logger.warning(f"Silero VAD error: {e}")
            return False

    def _transcribe_chunk(self, audio_data: np.ndarray) -> str:
        """Blocking transcription call. Run in thread via asyncio.to_thread."""
        segments, _info = self.transcribe_model.transcribe(
            audio_data,
            language=self.language,
            beam_size=1,
            condition_on_previous_text=False,
            vad_filter=False,
        )
        return " ".join(s.text.strip() for s in segments).strip()

    async def _run_transcribe_and_callback(self, buffer_copy: np.ndarray) -> None:
        """Run transcription in a worker thread and call user callback safely."""
        try:
            text = await asyncio.to_thread(self._transcribe_chunk, buffer_copy)
        except Exception as e:
            self.logger.exception(f"Transcription failed: {e}")
            return

        if not text:
            return

        if not self.on_transcript:
            return

        try:
            if asyncio.iscoroutinefunction(self.on_transcript):
                await self.on_transcript(text)
            else:
                await asyncio.to_thread(self.on_transcript, text)
        except Exception as e:
            self.logger.exception(f"on_transcript callback failed: {e}")

    # -------------------------
    # PyAudio callback -> async queue
    # -------------------------
    def audio_callback(self, in_data, frame_count, time_info, status):
        """Runs in PyAudio native thread. Put chunk into asyncio.Queue via loop.call_soon_threadsafe."""
        if status:
            self.logger.debug(f"Audio callback status: {status}")
        audio = np.frombuffer(in_data, dtype=np.int16).astype(np.float32) / 32768.0

        # the audio stream starts only after start() sets self.loop and self.async_queue
        if self.loop and self.async_queue:
            # safe: put_nowait from the audio thread via call_soon_threadsafe
            self.loop.call_soon_threadsafe(self.async_queue.put_nowait, audio)
        else:
            # fallback: drop audio if not ready
            self.logger.debug("Dropping audio: loop or async_queue not ready")

        return in_data, pyaudio.paContinue

    # -------------------------
    # Main async processing loop
    # -------------------------
    async def process_audio(self):
        """Consume audio chunks from self.async_queue and run VAD/chunking with minimal nesting."""
        assert self.async_queue is not None

        while self.is_running:
            # await next chunk (clean, no busy-sleep)
            try:
                chunk: np.ndarray = await self.async_queue.get()
            except asyncio.CancelledError:
                break

            is_speech = self._detect_speech_silero(chunk)

            if is_speech:
                # start or extend a speech buffer (we only append speech chunks)
                if not self.is_speech_active:
                    # You just STARTED talking - include pre-buffer to capture beginning
                    self.is_speech_active = True
                    self.speech_chunks = self.pre_buffer.copy() + [chunk.copy()]    # Pre-buffer + current chunk
                    self.speech_samples = sum(len(c) for c in self.speech_chunks)   # Count all samples
                    self.silence_counter = 0                # Reset silence timer
                    self.pre_buffer = []                    # Clear pre-buffer
                else:
                    # You're STILL talking
                    self.speech_chunks.append(chunk)        # Add to buffer
                    self.speech_samples += len(chunk)       # Count more samples
                    self.silence_counter = 0                # Reset silence timer
                continue    # Skip to next chunk

            # chunk is silence
            if not self.is_speech_active:
                # You're still quiet - add to pre-buffer (rolling buffer)
                self.pre_buffer.append(chunk.copy())
                if len(self.pre_buffer) > self.pre_buffer_chunks:
                    self.pre_buffer.pop(0)  # Remove oldest chunk
                continue    # Skip to next chunk

            # we were in speech; got a silence chunk -> count it
            self.silence_counter += len(chunk)

            if self.silence_counter < self.min_silence_samples:
                # not enough silence yet to finalize
                continue

            # enough silence observed -> finalize this speech segment
            if self.speech_samples >= self.min_speech_samples:

                # 1. Combine all speech chunks into one audio buffer
                if len(self.speech_chunks) > 1:
                    buffer_copy = np.concatenate(self.speech_chunks)  # Multiple chunks
                else:
                    buffer_copy = self.speech_chunks[0].copy()        # Single chunk
                
                # 2. IMMEDIATELY reset state (so we can capture new speech)
                self.is_speech_active = False
                self.speech_chunks = []
                self.speech_samples = 0
                self.silence_counter = 0

                # 3. Send to Whisper (in background, don't wait for it)
                asyncio.create_task(self._run_transcribe_and_callback(buffer_copy))
            else:
                # Speech was too short (< 250ms), just ignore it
                self.is_speech_active = False
                self.speech_chunks = []
                self.speech_samples = 0
                self.silence_counter = 0

    # -------------------------
    # Start / stop helpers
    # -------------------------
    async def start(self):
        """Start audio stream and processing loop. Returns when process_audio finishes (stop() called)."""
        if self.is_running:
            return

        self.loop = asyncio.get_running_loop()
        self.async_queue = asyncio.Queue()
        self.is_running = True

        # start pyaudio stream
        self._pyaudio = pyaudio.PyAudio()
        self._stream = self._pyaudio.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self.sample_rate,
            input=True,
            frames_per_buffer=512,
            stream_callback=self.audio_callback,
        )
        self._stream.start_stream()

        try:
            await self.process_audio()
        finally:
            # cleanup
            try:
                self._stream.stop_stream()
                self._stream.close()
            except Exception:
                pass
            try:
                self._pyaudio.terminate()
            except Exception:
                pass
            self.is_running = False
            self.loop = None
            self.async_queue = None

    async def stop(self, transcribe_remaining: bool = True):
        """Signal the processing loop to stop and optionally transcribe remaining speech."""
        self.is_running = False
        
        # If we have buffered speech and want to transcribe it
        if transcribe_remaining and self.is_speech_active and len(self.speech_chunks) > 0:
            # Combine speech chunks
            if len(self.speech_chunks) > 1:
                buffer_copy = np.concatenate(self.speech_chunks)
            else:
                buffer_copy = self.speech_chunks[0].copy()
            
            # Clear state
            self.is_speech_active = False
            self.speech_chunks = []
            self.speech_samples = 0
            self.silence_counter = 0
            
            # Transcribe immediately (wait for completion)
            await self._run_transcribe_and_callback(buffer_copy)
        
        # Wait for process_audio to exit
        await asyncio.sleep(0.05)


## 🧪 Live Transcription Test

This test starts a 10-second live recording session using the `LiveTranscriber`.  
Speak naturally in short sentences — each pause will automatically trigger a transcription.  
Each transcribed chunk is printed as soon as it’s ready, and all results are shown at the end.


In [ ]:
#| eval: false
import asyncio

all_chunks = []

def handle_transcript_chunk(text: str):
    """Callback called whenever a transcription chunk is ready."""
    if text.strip():
        print(f"\n[TRANSCRIBED] {text}")
        all_chunks.append(text)

async def test_live_transcription(duration_seconds: int = 10):
    all_chunks.clear()
    print("🎤 Speak in short sentences; pauses will trigger transcription.")
    transcriber = LiveTranscriber(
        model_id="base",          # keep whatever model id you use
        language="en",
        on_transcript=handle_transcript_chunk,
        vad_threshold=0.5,
        min_speech_duration_ms=250,
        min_silence_duration_ms=500,
    )

    # start the transcriber in the background
    start_task = asyncio.create_task(transcriber.start())

    try:
        # run for given duration (you can interrupt with Ctrl+C)
        await asyncio.sleep(duration_seconds)
    except KeyboardInterrupt:
        print("Interrupted by user.")
    finally:
        # ask the transcriber to stop and wait for it to finish
        await transcriber.stop()

        # wait for the start() task to exit cleanly
        try:
            await start_task
        except Exception as e:
            # if any error bubbled up from start/process_audio, show it
            print(f"Transcriber task ended with exception: {e}")

    print("\n📝 Full transcript:")
    for i, t in enumerate(all_chunks, 1):
        print(f"{i}. {t}")

# run it
await test_live_transcription(10)



🎤 Speak in short sentences; pauses will trigger transcription.


Using cache found in /home/jens/.cache/torch/hub/snakers4_silero-vad_master
ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave



[TRANSCRIBED] Okay, can you hear me?

[TRANSCRIBED] That's nice.

[TRANSCRIBED] What if now?

[TRANSCRIBED] and now

📝 Full transcript:
1. Okay, can you hear me?
2. That's nice.
3. What if now?
4. and now
